# Data Load

In [1]:
import config
from pshmodule.utils import filemanager as fm

In [2]:
df = fm.load(config.temp_data)

extension : .pickle
Loaded 10317 records from /Volumes/GoogleDrive/내 드라이브/MemeProject/data/temp_for_doc2vec.pickle


In [3]:
df.head()

,content,pos
0,나 중간고사 반에서 1등했어,"[나, 중간고사, 반]"
1,중간고사 점수 내가 반에서 제일 잘 받음,"[중간고사, 점수, 내, 반, 제일, 받음]"
2,나 반에서 중간 성적 제일 좋아,"[나, 반, 중간, 성적, 제일, 좋아]"
3,우리 반에서 내가 시험 제일 잘 봤다,"[우리, 반, 내, 시험, 제일, 봤다]"
4,중간고사 반 1등 먹음,"[중간고사, 반, 먹음]"


# 

# Embedding

In [4]:
import swifter
import nltk
import torch
import torch.nn as nn

In [5]:
device = torch.device('mps:0' if torch.backends.mps.is_available() else 'cpu')
print(device)

mps:0


### make vocabulary

In [6]:
tokenized = []

for i in df.pos.tolist():
    for j in i:
        tokenized.append(j)

In [7]:
vocab = set(tokenized)

In [8]:
print(len(vocab))

8286


### indexing

In [9]:
word2index={}
for voca in vocab:
    if word2index.get(voca)==None:
        word2index[voca]=len(word2index)

In [10]:
print(word2index)

{'이름': 0, '깎아': 1, '아저씨': 2, '학원': 3, '올리': 4, '연인': 5, '쌤': 6, '픔': 7, '밤샘': 8, '도랐음': 9, '공한': 10, '런닝머신': 11, '필요한': 12, '부속': 13, '케텍스': 14, '딴딴하고': 15, '버전': 16, '날씬해질': 17, '있네': 18, '좔좔': 19, '야간': 20, '지각': 21, '솔트': 22, '늘리면': 23, '줄여라': 24, '덜어내야지': 25, '시중': 26, '훅': 27, '의학': 28, '타가': 29, '마시려고': 30, '고등어': 31, '화장': 32, '중이': 33, '주스': 34, '가벼워져': 35, '모른다는': 36, '면상': 37, '만나기로': 38, '할려고': 39, '걸어서만': 40, '기차': 41, '싫증난': 42, '쎈': 43, '재료': 44, '수입': 45, '세대': 46, '얼마나': 47, '어째서': 48, '낮잠': 49, '벌어지지': 50, '크레이지': 51, '수호': 52, '괜': 53, '태블릿': 54, '카네이션': 55, '날아다님': 56, '비켜': 57, '좋지': 58, '뜨겁다': 59, '넓어지길': 60, '호텔': 61, '아끼는': 62, '가라': 63, '처피뱅': 64, '예뻐졌어': 65, '휴학': 66, '대중교통': 67, '좋아하지': 68, '조켔다': 69, '딸림': 70, '사라지더라': 71, '겨울': 72, '흡입': 73, '농락': 74, '숫자': 75, '귀신': 76, '야옹이': 77, '왔어요': 78, '사촌': 79, '대감': 80, '얻을': 81, '레토': 82, '였음': 83, '어휘': 84, '블루투스': 85, '웃겼어': 86, '렉스': 87, '돌았는데': 88, '닫으셈': 89, '디네': 90, '닫냐': 91, '활동': 92, '보내지': 93, '태우러': 94, 

### make data

In [11]:
windows = list(nltk.ngrams(['<DUMMY>'] * config.window_size + tokenized + ['<DUMMY>'] * config.window_size, config.window_size * 2 + 1))


In [12]:
print(windows)

[('<DUMMY>', '<DUMMY>', '나', '중간고사', '반'), ('<DUMMY>', '나', '중간고사', '반', '중간고사'), ('나', '중간고사', '반', '중간고사', '점수'), ('중간고사', '반', '중간고사', '점수', '내'), ('반', '중간고사', '점수', '내', '반'), ('중간고사', '점수', '내', '반', '제일'), ('점수', '내', '반', '제일', '받음'), ('내', '반', '제일', '받음', '나'), ('반', '제일', '받음', '나', '반'), ('제일', '받음', '나', '반', '중간'), ('받음', '나', '반', '중간', '성적'), ('나', '반', '중간', '성적', '제일'), ('반', '중간', '성적', '제일', '좋아'), ('중간', '성적', '제일', '좋아', '우리'), ('성적', '제일', '좋아', '우리', '반'), ('제일', '좋아', '우리', '반', '내'), ('좋아', '우리', '반', '내', '시험'), ('우리', '반', '내', '시험', '제일'), ('반', '내', '시험', '제일', '봤다'), ('내', '시험', '제일', '봤다', '중간고사'), ('시험', '제일', '봤다', '중간고사', '반'), ('제일', '봤다', '중간고사', '반', '먹음'), ('봤다', '중간고사', '반', '먹음', '중간'), ('중간고사', '반', '먹음', '중간', '시험'), ('반', '먹음', '중간', '시험', '내'), ('먹음', '중간', '시험', '내', '반'), ('중간', '시험', '내', '반', '너'), ('시험', '내', '반', '너', '왜'), ('내', '반', '너', '왜', '이렇게'), ('반', '너', '왜', '이렇게', '멍청하냐'), ('너', '왜', '이렇게', '멍청하냐', '새끼'), ('왜', '이렇게', '멍청하냐'